In [45]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
from mtflearn.io import load_image
from stemplot import imshow, colors_from_lbs
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints
import re
from skimage.transform import resize

Load images, pts and lbs

In [56]:
size_dict = {
            'Ti':{'1028_ADF1_2':9,'1028_ADF1_11':8,'1028_ADF1_25':9,'0106_ADF1_1':17,'0106_ADF1_6':17,'0106_ADF1_8':18,'0106_ADF1_10':18,'0106_ADF1_11':21,'0106_ADF1_12':21},
            'V':{'E1_ADF1_3':14,'E1_ADF1_8':18,'E2_ADF1_3':19,'E2_ADF1_4':18,'E2_ADF1_5':14,'E2_ADF1_6':17,'E2_ADF1_7':17,'E2_ADF1_8':18,'E2_ADF1_14':26},
            'Mn':{'ADF1_2':22,'ADF1_4':18,'ADF1_7':18,'ADF1_12':22,'ADF1_18':21},
            'Co':{'ADF1_5':18,'ADF1_8':15,'ADF1_9':15,'ADF1_18':14,'ADF1_27':17,'ADF1_10':18,'ADF1_11':18}
            }

In [10]:
pattern = r'^\d{4}$|^[Ee]\d$'

# 使用 re.match 来尝试匹配字符串
if re.match(pattern, 'V5'):
    print(1)

In [37]:
import glob
def find_imgs(element):
    dm4_files = glob.glob(f"{element}/*.dm4")
    adf1_files = []
    adf2_files = []
    for file in dm4_files:
        if "ADF1" in file and "ADF2" not in file:
            adf1_files.append(file)
        elif "ADF2" in file and "ADF1" not in file:
            adf2_files.append(file)
    return adf1_files, adf2_files

def find_pts_and_lbs(element):
    folder_path = f"{element} pts and lbs"
    npy_files = glob.glob(f"{folder_path}/*.npy")
    pts_files = []
    lbs_files = []
    for file in npy_files:
        file_name = file.split("\\")[-1]
        if "pts" in file_name and "lbs" not in file_name:
            pts_files.append(file)
        elif "lbs" in file_name and "pts" not in file_name:
            lbs_files.append(file)
    return pts_files, lbs_files
                    
def find_matching_images(lbs_filenames,pts_filenames, adf1_filenames,adf2_filenames):
    matching_images = []

    for idx,lbs_file in enumerate(lbs_filenames):
        files = []
        files.append(lbs_file)
        files.append(pts_filenames[idx])
        file_name = lbs_file.split('\\')[-1]
        if re.match(r'^[0-9E]', file_name):
            base_name = file_name.split('_')[0]
            if base_name[0] == 'E':
                base_name = base_name.split(' ')[0]
        else:
            base_name = ''
        match = re.search(r'(Ti|V|Mn|Co)(\d+)', file_name)
        if match:
            element = match.group(1)
            number = match.group(2)
            number = number+'.dm4'
            for idx2,img_file in enumerate(adf1_filenames):
                if base_name and (base_name in img_file and number in img_file):
                    files.append(img_file)
                    files.append(adf2_filenames[idx2])
                    file_name = (img_file.split('\\')[1]).split('.')[0]
                    files.append(size_dict[element][file_name])
                    break
                elif not base_name and number in img_file:
                    files.append(img_file)
                    files.append(adf2_filenames[idx2])
                    file_name = (img_file.split('\\')[1]).split('.')[0]
                    files.append(size_dict[element][file_name])
                    break
        matching_images.append(files)
    return matching_images

In [52]:
Ti_adf1,Ti_adf2 = find_imgs('Ti')
Ti_pts,Ti_lbs = find_pts_and_lbs('Ti')
Ti_pair_files = find_matching_images(Ti_lbs,Ti_pts,Ti_adf1,Ti_adf2)

In [53]:
for info in Ti_pair_files:
    _,_,img1_file,img2_file,size = info
    img1 = load_image(img1_file)
    img2 = load_image(img2_file)
    file_name = (img1_file.split('\\')[1]).split('.')[0]
    if img1.shape[0] == 2048:
        img1 = resize(img1, (1024, 1024))
        img2 = resize(img2, (1024, 1024))
    print(file_name,get_characteristic_length(img1),get_characteristic_length(img2),size)

0106_ADF1_10 18 18 18
0106_ADF1_11 21 21 21
0106_ADF1_12 21 21 21
0106_ADF1_1 17 17 17
0106_ADF1_6 17 17 17
0106_ADF1_8 18 18 18
1028_ADF1_11 8 8 8
1028_ADF1_25 9 8 9
1028_ADF1_2 9 9 9


In [57]:
V_adf1,V_adf2 = find_imgs('V')
V_pts,V_lbs = find_pts_and_lbs('V')
V_pair_files = find_matching_images(V_lbs,V_pts,V_adf1,V_adf2)

In [58]:
for info in V_pair_files:
    _,_,img1_file,img2_file,size = info
    img1 = load_image(img1_file)
    img2 = load_image(img2_file)
    file_name = (img1_file.split('\\')[1]).split('.')[0]
    if img1.shape[0] == 2048:
        img1 = resize(img1, (1024, 1024))
        img2 = resize(img2, (1024, 1024))
    print(file_name,get_characteristic_length(img1),get_characteristic_length(img2),size)

E1_ADF1_3 14 14 14
E1_ADF1_8 18 18 18
E2_ADF1_14 26 26 26
E2_ADF1_3 18 18 19
E2_ADF1_14 26 26 26
E2_ADF1_5 14 14 14
E2_ADF1_6 17 17 17
E2_ADF1_7 17 17 17
E2_ADF1_8 17 17 18


In [41]:
Mn_adf1,Mn_adf2 = find_imgs('Mn')
Mn_pts,Mn_lbs = find_pts_and_lbs('Mn')
Mn_pair_files = find_matching_images(Mn_lbs,Mn_pts,Mn_adf1,Mn_adf2)

In [59]:
for info in Mn_pair_files:
    _,_,img1_file,img2_file,size = info
    img1 = load_image(img1_file)
    img2 = load_image(img2_file)
    file_name = (img1_file.split('\\')[1]).split('.')[0]
    if img1.shape[0] == 2048:
        img1 = resize(img1, (1024, 1024))
        img2 = resize(img2, (1024, 1024))
    print(file_name,get_characteristic_length(img1),get_characteristic_length(img2),size)

ADF1_12 21 21 22
ADF1_18 21 21 21
ADF1_12 21 21 22
ADF1_4 17 17 18
ADF1_7 17 17 18


In [42]:
Co_adf1,Co_adf2 = find_imgs('Co')
Co_pts,Co_lbs = find_pts_and_lbs('Co')
Co_pair_files = find_matching_images(Co_lbs,Co_pts,Co_adf1,Co_adf2)

In [60]:
for info in Co_pair_files:
    _,_,img1_file,img2_file,size = info
    img1 = load_image(img1_file)
    img2 = load_image(img2_file)
    file_name = (img1_file.split('\\')[1]).split('.')[0]
    if img1.shape[0] == 2048:
        img1 = resize(img1, (1024, 1024))
        img2 = resize(img2, (1024, 1024))
    print(file_name,get_characteristic_length(img1),get_characteristic_length(img2),size)

ADF1_10 18 18 18
ADF1_11 18 18 18
ADF1_18 14 14 14
ADF1_27 17 17 17
ADF1_5 18 18 18
ADF1_18 14 14 14
ADF1_9 15 15 15
